In [1]:
import import_ipynb
import random
import sensors
from environment import Environment

importing Jupyter notebook from sensors.ipynb
importing Jupyter notebook from environment.ipynb
Sofa


In [2]:
# As per slide 52, it receives the environment, uses simple rules (here random) to chose action (here it is moving to a position)
class simple_reflex_agent:
    def act(self,environ):
        # First it checks which positions dirty
        dirty = []
        for i in range(len(environ)):
            for j in range(len(environ[i])):
                if environ[i][j]:
                    dirty.append([i,j])

        if len(dirty) == 0:
            print("No dirt found by sensor (ie. in 2 unit distance)")
            return [0,0]

        loc = random.choice(dirty)
        
        return [ loc[0] - 2, loc[1] - 2 ]

class VacuumCleaner:
    # Performance score is based on how much dirty region we could detect, no other complexity for now
    def __init__(self):
        environ = Environment()
        # Considering a rectangle we only need TOP-LEFT and BOTTOM-RIGHT to know all corners/edges
        tl_corner = [ 0, len(environ.environ)-1 ]
        br_corner = [ len(environ.environ[0])-1, 0 ]

        self.environ = environ
        self.position_sensor = sensors.PositionSensor( tl_corner, br_corner, position=[2,2] )
        self.visual_sensor = sensors.VisualSensor()
        self.perf = 0
        print("Initial State: ")
        self._debug_print_environ()
    
    def get_environment(self):
        return self.environ
    
    def get_curr_pos(self):
        return self.position_sensor.position;

    def attempt_clean(self,agent = simple_reflex_agent()):
        # will always return 5*5 matrix
        locations = self.visual_sensor.scan_dirty_regions(self)
        _perf = 0
        for row in locations:
            for b in row:
                if b:
                    _perf += 1

        # It simply always increases, but the more dirt it finds, the more quickly it increases.
        self.perf += _perf / 25
        
        # ACTUATOR
        # We pass a limited envionment to Reflex agent
        move = agent.act(locations)

        # not logically correct for sensor to do move i think, but for now it will work
        self.position_sensor.move(move)

        new_pos = self.get_curr_pos()
        if self.environ.environ[ new_pos[0] ][ new_pos[1] ].is_dirty:
            print("Cleaning: ", new_pos)
            self.environ.environ[ new_pos[0] ][ new_pos[1] ].is_dirty = False
            self.perf += len(locations)*0.4;  # TODO: perf += dirt_found/total_boxes
            self._debug_print_environ()
        else:
            print("Already clean: ", new_pos)

    def _debug_print_environ(self):
        mat = self.environ.environ
        
        for i in range(len(mat)):
            for j in range(len(mat)):
                if mat[i][j].is_dirty:
                    print("*", end=" ")
                else:
                    print(" ", end=" ")
            print()

In [3]:
VacuumCleaner

__main__.VacuumCleaner